In [29]:
import folium
from branca.element import MacroElement
from jinja2 import Template

# === 1. 도로 위험도 데이터 ===
road_data = [
    {"lat": 37.5, "lon": 127.0, "risk_score": 90, "label": "교차로/신호등 다수"},
    {"lat": 37.51, "lon": 127.01, "risk_score": 65, "label": "차로 수 많음"},
    {"lat": 37.52, "lon": 127.02, "risk_score": 35, "label": "평균속도 낮음"},
    {"lat": 37.53, "lon": 127.03, "risk_score": 15, "label": "단일로"},
]

# === 2. 위험도에 따라 색상 결정 ===
def get_color(score):
    if score > 70:
        return "red"
    elif score > 40:
        return "orange"
    else:
        return "green"

# === 3. 위험도에 따라 기본 반지름 설정 ===
def get_base_radius(score):
    if score > 70:
        return 12
    elif score > 40:
        return 8
    else:
        return 5

# === 4. 지도 생성 ===
m = folium.Map(location=[37.51, 127.015], zoom_start=13)
circle_js_array = []

# === 5. 도로 위험 마커 추가 ===
for idx, road in enumerate(road_data):
    radius = get_base_radius(road["risk_score"])
    color = get_color(road["risk_score"])

    marker = folium.CircleMarker(
        location=[road["lat"], road["lon"]],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=road["label"]
    ).add_to(m)

    marker._name = f"circle_{idx}"
    circle_js_array.append(f"circle_{idx}")

# === 6. 줌에 따라 크기 조절되는 JavaScript 코드 생성 ===
circle_updates = ""
for name in circle_js_array:
    circle_updates += f"""
        var {name} = circles['{name}'];
        if ({name}) {{
            var newRadius = {name}.options.baseRadius * (map.getZoom() / 12);
            {name}.setRadius(newRadius);
        }}
    """

zoom_js = f"""
<script>
    function updateCircleSizes() {{
        {circle_updates}
    }}
    map.on('zoomend', updateCircleSizes);
    updateCircleSizes();
</script>
"""

init_js = "<script>var circles = {};</script>"
attach_js = ""
for i in range(len(road_data)):
    attach_js += f"""
    circles["circle_{i}"] = circle_{i};
    circles["circle_{i}"].options.baseRadius = circles["circle_{i}"].getRadius();
    """

# === 7. JS 삽입용 클래스 ===
class BindJS(MacroElement):
    def __init__(self, html):
        super().__init__()
        self._template = Template(f"""
        {{% macro script(this, kwargs) %}}
        {html}
        {{% endmacro %}}
        """)

# === 8. 지도에 JS 붙이기 ===
m.get_root().add_child(BindJS(init_js + f"<script>{attach_js}</script>" + zoom_js))

# === 9. HTML 저장 ===
m.save("road_risk_zoom_map.html")
print("✅ 지도 저장 완료: road_risk_zoom_map.html")

✅ 지도 저장 완료: road_risk_zoom_map.html


In [ ]:
import folium
import pandas as pd
from branca.element import MacroElement
from jinja2 import Template

# === 1. 데이터 불러오기 ===
df = pd.read_csv("/Users/leejuan/Documents/GitHub/senior-road-risk-analyzer/scripts/road_risk_data.csv")  # 경로는 필요 시 조정

# === 2. 색상 설정 함수 ===
def get_color(score):
    if score > 70:
        return "red"
    elif score > 40:
        return "orange"
    else:
        return "green"

# === 3. 반지름 설정 함수 ===
def get_base_radius(score):
    if score > 70:
        return 12
    elif score > 40:
        return 8
    else:
        return 5

# === 4. 지도 생성 ===
m = folium.Map(location=[df["lat"].mean(), df["lon"].mean()], zoom_start=13)
circle_js_array = []

# === 5. 마커 추가 ===
for idx, row in df.iterrows():
    radius = get_base_radius(row["risk_score"])
    color = get_color(row["risk_score"])

    marker = folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=row.get("label", "위험 구간")
    ).add_to(m)

    marker._name = f"circle_{idx}"
    circle_js_array.append(f"circle_{idx}")

# === 6. 줌 레벨에 따라 반지름 동적 조정 JS ===
circle_updates = ""
for name in circle_js_array:
    circle_updates += f"""
        var {name} = circles['{name}'];
        if ({name}) {{
            var newRadius = {name}.options.baseRadius * (map.getZoom() / 12);
            {name}.setRadius(newRadius);
        }}
    """

zoom_js = f"""
<script>
    function updateCircleSizes() {{
        {circle_updates}
    }}
    map.on('zoomend', updateCircleSizes);
    updateCircleSizes();
</script>
"""

# === 7. JS 변수 초기화 및 바인딩 ===
init_js = "<script>var circles = {};</script>"
attach_js = ""
for i in range(len(df)):
    attach_js += f"""
    circles["circle_{i}"] = circle_{i};
    circles["circle_{i}"].options.baseRadius = circles["circle_{i}"].getRadius();
    """

class BindJS(MacroElement):
    def __init__(self, html):
        super().__init__()
        self._template = Template(f"""
        {{% macro script(this, kwargs) %}}
        {html}
        {{% endmacro %}}
        """)

m.get_root().add_child(BindJS(init_js + f"<script>{attach_js}</script>" + zoom_js))

# === 8. 저장 ===
m.save("/Users/leejuan/Documents/GitHub/senior-road-risk-analyzer/scripts/road_risk_zoom_map.html")
print("✅ 시각화 지도 저장 완료: scripts/road_risk_zoom_map.html")

FileNotFoundError: [Errno 2] No such file or directory: 'scripts/road_risk_zoom_map.html'

In [11]:
import folium
import pandas as pd
from branca.element import MacroElement
from jinja2 import Template

# === 1. 데이터 불러오기 ===
df = pd.read_csv("/Users/leejuan/Documents/GitHub/senior-road-risk-analyzer/scripts/road_risk_data.csv")  # 경로는 필요 시 조정

# === 2. 색상 설정 함수 ===
def get_color(score):
    if score > 70:
        return "red"
    elif score > 40:
        return "orange"
    else:
        return "green"

# === 3. 반지름 설정 함수 ===
def get_base_radius(score):
    if score > 70:
        return 12
    elif score > 40:
        return 8
    else:
        return 5

# === 4. 지도 생성 ===
m = folium.Map(location=[df["lat"].mean(), df["lon"].mean()], zoom_start=13)
circle_js_array = []

# === 5. 마커 추가 ===
for idx, row in df.iterrows():
    radius = get_base_radius(row["risk_score"])
    color = get_color(row["risk_score"])

    marker = folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=row.get("label", "위험 구간")
    ).add_to(m)

    marker._name = f"circle_{idx}"
    circle_js_array.append(f"circle_{idx}")

# === 6. 줌 레벨에 따라 반지름 동적 조정 JS ===
circle_updates = ""
for name in circle_js_array:
    circle_updates += f"""
        var {name} = circles['{name}'];
        if ({name}) {{
            var newRadius = {name}.options.baseRadius * (map.getZoom() / 12);
            {name}.setRadius(newRadius);
        }}
    """

zoom_js = f"""
<script>
    function updateCircleSizes() {{
        {circle_updates}
    }}
    map.on('zoomend', updateCircleSizes);
    updateCircleSizes();
</script>
"""

# === 7. JS 변수 초기화 및 바인딩 ===
init_js = "<script>var circles = {};</script>"
attach_js = ""
for i in range(len(df)):
    attach_js += f"""
    circles["circle_{i}"] = circle_{i};
    circles["circle_{i}"].options.baseRadius = circles["circle_{i}"].getRadius();
    """

class BindJS(MacroElement):
    def __init__(self, html):
        super().__init__()
        self._template = Template(f"""
        {{% macro script(this, kwargs) %}}
        {html}
        {{% endmacro %}}
        """)

m.get_root().add_child(BindJS(init_js + f"<script>{attach_js}</script>" + zoom_js))

# === 8. 저장 ===
m.save("/Users/leejuan/Documents/GitHub/senior-road-risk-analyzer/scripts/road_risk_zoom_map.html")
print("✅ 시각화 지도 저장 완료: scripts/road_risk_zoom_map.html")

✅ 시각화 지도 저장 완료: scripts/road_risk_zoom_map.html


In [21]:
import requests
import folium

# Kakao API 요청
url = "https://apis-navi.kakaomobility.com/v1/directions"
headers = {
    "Authorization": "KakaoAK fd5cd52e7afcde28a29cb81d157fb92e"  # ← 본인 REST 키
}
params = {
    "origin": "126.97843,37.56668",       # 출발: 서울시청
    "destination": "127.10560,37.35957",  # 도착: 판교
    "priority": "RECOMMEND"
}

res = requests.get(url, headers=headers, params=params)
data = res.json()
print(data)

{'trans_id': '0196cdeac399781497f4c97e1279cf85', 'routes': [{'result_code': 0, 'result_msg': '길찾기 성공', 'summary': {'origin': {'name': '', 'x': 126.97841971428976, 'y': 37.566672250196135}, 'destination': {'name': '', 'x': 127.1055988344799, 'y': 37.35956713687521}, 'waypoints': [], 'priority': 'RECOMMEND', 'bound': {'min_x': 126.98024071029057, 'min_y': 37.3478570834374, 'max_x': 127.10349459450194, 'max_y': 37.57012114430578}, 'fare': {'taxi': 40800, 'toll': 1000}, 'distance': 31228, 'duration': 5415}, 'sections': [{'distance': 31227, 'duration': 5414, 'bound': {'min_x': 127.10581618350132, 'min_y': 37.348835896670934, 'max_x': 127.10600133755209, 'max_y': 37.56909722870951}, 'roads': [{'name': '', 'distance': 418, 'duration': 94, 'traffic_speed': 16.0, 'traffic_state': 0, 'vertexes': [126.97847261455232, 37.56694302037848, 126.97847261455232, 37.56694302037848, 126.97904997140229, 37.56693900440904, 126.97905661155959, 37.56645249943343, 126.97825720085571, 37.56613922932406, 126.977

In [18]:
# 경로에서 vertexes 추출
vertexes = data['routes'][0]['sections'][0]['roads']

# 모든 경로 포인트 담을 리스트
points = []

for road in vertexes:
    coords = road['vertexes']
    # [x1, y1, x2, y2, ...] → (lat, lon) 쌍으로
    latlon = [(coords[i+1], coords[i]) for i in range(0, len(coords), 2)]
    points.extend(latlon)

In [ ]:
# folium 지도 초기화 (출발지 기준)
start_latlon = points[0]
m = folium.Map(location=start_latlon, zoom_start=12)

# 경로 PolyLine 추가
folium.PolyLine(points, color="blue", weight=5, opacity=0.8).add_to(m)

# 출발/도착 마커
folium.Marker(points[0], tooltip="출발지", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(points[-1], tooltip="도착지", icon=folium.Icon(color="red")).add_to(m)

# HTML 파일로 저장
m.save("/Users/leejuan/Documents/GitHub/senior-road-risk-analyzer/scripts/kakao_route_map.html")
print("✅ 지도 저장 완료: kakao_route_map.html")

✅ 지도 저장 완료: kakao_route_map.html


In [23]:
import requests
import folium

# 1. Kakao 길찾기 요청
url = "https://apis-navi.kakaomobility.com/v1/directions"
headers = {
    "Authorization": "KakaoAK fd5cd52e7afcde28a29cb81d157fb92e"  # REST API 키
}
params = {
    "origin": "126.97843,37.56668",       # 서울시청
    "destination": "127.10560,37.35957",  # 판교
    "priority": "RECOMMEND"
}

res = requests.get(url, headers=headers, params=params)
data = res.json()

# 2. vertexes 추출 및 위경도 변환
vertexes = data['routes'][0]['sections'][0]['roads']
points = []

for road in vertexes:
    coords = road['vertexes']
    latlon = [(coords[i+1], coords[i]) for i in range(0, len(coords), 2)]
    points.extend(latlon)

# 3. 지도 생성
m = folium.Map(location=points[0], zoom_start=12)

# 4. PolyLine 경로 표시
folium.PolyLine(points, color="blue", weight=5, opacity=0.8).add_to(m)

# 5. 지점마다 위도·경도 툴팁 마커 표시
for lat, lon in points:
    folium.CircleMarker(
        location=(lat, lon),
        radius=3,
        color="red",
        fill=True,
        fill_opacity=0.7,
        tooltip=f"위도: {lat:.5f}, 경도: {lon:.5f}"
    ).add_to(m)

# 6. 출발/도착 마커
folium.Marker(points[0], tooltip="출발지", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(points[-1], tooltip="도착지", icon=folium.Icon(color="red")).add_to(m)

# 7. 저장
m.save("/Users/leejuan/Documents/GitHub/senior-road-risk-analyzer/scripts/kakao_route_with_coords.html")
print("✅ 지도 저장 완료: kakao_route_with_coords.html")

✅ 지도 저장 완료: kakao_route_with_coords.html


In [24]:
import requests
import folium
import random

# Kakao 길찾기 요청
url = "https://apis-navi.kakaomobility.com/v1/directions"
headers = {
    "Authorization": "KakaoAK fd5cd52e7afcde28a29cb81d157fb92e"
}
params = {
    "origin": "126.97843,37.56668",       # 서울시청
    "destination": "127.10560,37.35957",  # 판교
    "priority": "RECOMMEND"
}

res = requests.get(url, headers=headers, params=params)
data = res.json()

# 지도 초기화
m = folium.Map(location=[37.56668, 126.97843], zoom_start=12)

# 로드별 시각화
roads = data['routes'][0]['sections'][0]['roads']
color_list = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'black']

for i, road in enumerate(roads):
    road_name = road['name'] if road['name'] else f"Unnamed-{i}"
    coords = road['vertexes']
    latlon = [(coords[i+1], coords[i]) for i in range(0, len(coords), 2)]

    folium.PolyLine(
        latlon,
        color=random.choice(color_list),
        weight=5,
        opacity=0.7,
        tooltip=road_name
    ).add_to(m)

# 출발지 / 도착지 마커
start = roads[0]['vertexes']
end = roads[-1]['vertexes']
start_coord = (start[1], start[0])
end_coord = (end[-1], end[-2])

folium.Marker(start_coord, tooltip="출발지", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(end_coord, tooltip="도착지", icon=folium.Icon(color="red")).add_to(m)

# 저장
m.save("/Users/leejuan/Documents/GitHub/senior-road-risk-analyzer/scripts/kakao_route_by_roadname.html")
print("✅ 도로 구간별 지도 저장 완료: kakao_route_by_roadname.html")

✅ 도로 구간별 지도 저장 완료: kakao_route_by_roadname.html
